<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.0


    Uninstalling protobuf-6.31.0:
      Successfully uninstalled protobuf-6.31.0


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.61


    Uninstalling yfinance-0.2.61:
      Successfully uninstalled yfinance-0.2.61
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))
  print('-------------------')
  print('Est LTT: ',  format_amt(estimate_lt))
  print('Est LTT PnL: '+ '{} ({}%)'.format(format_amt(est_lt_pnl_amount), est_lt_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')

Run date time (IST): 2025-05-26 11:20:28
-------------------
qualified stocks: 94
with latest results: 32
still star stocks: 18
-------------------
Initial Investment:  1.14 C
CY Investment:  1.41 C
Reserve:  1.87 L
Current:  1.29 C
-------------------
Today PnL: 56.56 K (0.44%)
Current PnL: -16.66 L (-11.81%)
CY Booked + Current PnL: -12.37 L (-8.77%)
-------------------
Total profit:  3.81 L
Total loss:  -20.47 L
-------------------
Total Booked + Current PnL: 14.61 L (12.81%)
Total Booked PnL: 31.27 L (27.42%)
Curr Year Booked PnL: 4.29 L (3.32%)
Prev Year Booked PnL: 26.98 L (23.66%)
Est FTT:  2.09 C
Est FTT PnL: 79.64 L (61.69%)
-------------------
Est LTT:  2.70 C
Est LTT PnL: 1.41 C (109.21%)
Deployed:  1.14 C
Current:  1.29 C
CAGR/XIRR %: 8.8%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SBILIFE,1496.49,1924.99,1.57,H-LC,44.68,100621.0,16818.0,7174.0,-0.11,20.07,7.13,28.63,42.0,2.34,0.79,34.46,AR,ATH,INSURANCE
62,RELIANCE,1291.83,1526.00,5.36,X-LC,52.13,156775.0,15966.0,9563.0,0.81,11.34,6.10,18.13,17.0,1.67,1.23,23.79,XY25,NTT,REFINERIES
4,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,91.49,99641.0,21279.0,12754.0,0.57,27.15,12.80,43.43,71.0,1.67,0.78,33.27,XY24,NTT,MISC
56,PIDILITIND,2504.06,3576.00,-17.12,X-LC,28.72,91905.0,16783.0,15376.0,1.05,22.34,16.73,42.81,11.0,1.09,0.72,15.26,X40,BTT,CHEMICALS
69,SBIN,760.30,863.00,-14.39,M-LC,14.89,210211.0,8732.0,18478.0,0.35,4.33,8.79,13.51,73.0,0.47,1.65,17.48,XY25,NTT,BANKS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,VAIBHAVGBL,350.21,521.00,69.70,X-SC,89.36,113248.0,-38743.0,112863.0,7.90,-25.49,99.66,48.77,59.0,-0.34,0.89,30.08,XR,NTT,JEWELLERY
7,ASIANTILES,75.41,137.00,5655.56,M-SC,90.43,63158.0,-30652.0,107268.0,2.69,-32.67,169.84,81.67,203.0,-0.29,0.50,22.31,XR,NTT,CERAMICS
5,ALKYLAMINE,2347.98,4567.52,-13.09,X-SC,51.06,80874.0,-20089.0,115529.0,2.49,-19.90,142.85,94.53,60.0,-0.17,0.64,19.90,SR,ATH,CHEMICALS
18,CERA,8421.60,10420.45,-17.90,X-SC,98.94,84682.0,-24799.0,50784.0,2.33,-22.65,59.97,23.73,64.0,-0.49,0.67,28.17,X40N,ATH,CERAMICS
52,MASFIN,326.60,399.50,-21.86,H-SC,64.89,88680.0,-9300.0,31171.0,2.28,-9.49,35.15,22.32,130.0,-0.30,0.70,28.62,XR,ATH,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
20,COFFEEDAY,59.14,80.00,-57.75,L-SC,47.87,61690.0,-51859.0,91912.0,-4.20,-45.67,148.99,35.27,238.0,-0.56,0.48,50.99,XR,NTT,HOTELS
51,LUXIND,1448.19,2442.00,41.05,M-SC,77.66,146158.0,8580.0,85839.0,-2.95,6.24,58.73,68.62,182.0,0.10,1.15,26.71,X40N,NTT,TEXTILES
36,ICICIGI,1839.64,2268.71,-17.01,H-MC,23.40,136856.0,723.0,31025.0,-2.41,0.53,22.67,23.32,96.0,0.02,1.08,21.17,X40,ATH,INSURANCE
75,STARHEALTH,551.56,761.00,0.76,M-SC,87.23,164820.0,-35396.0,111418.0,-1.94,-17.68,67.60,37.97,194.0,-0.32,1.30,32.34,XY24,NTT,INSURANCE
38,INDIAMART,2327.09,4911.36,-52.59,H-SC,46.81,123410.0,74.0,136886.0,-1.67,0.06,110.92,111.05,108.0,0.00,0.97,20.92,AR,ATH,MISC


In [12]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
41,INFY,1450.31,2275.00,-12.06,X-LC,59.04,251061.0,20462.0,110668.0,0.91,8.87,44.08,56.86,1.0,0.18,1.97,14.59,X40,BTT,IT
81,TCS,3841.06,4998.00,-15.19,X-LC,40.43,244329.0,-20704.0,100541.0,0.75,-7.81,41.15,30.12,2.0,-0.21,1.92,9.58,X40,BTT,IT
50,LTIM,5564.16,7197.33,-4.53,X-LC,88.30,183830.0,-16480.0,75278.0,1.20,-8.23,40.95,29.35,4.0,-0.22,1.44,27.00,X200,ATH,IT
72,SIEMENS,4406.90,7969.85,-9.91,X-LC,81.91,81482.0,-28690.0,117766.0,-0.55,-26.04,144.53,80.85,5.0,-0.24,0.64,20.01,AR,ATH,ELECTRICAL
32,HAVELLS,1593.67,2069.16,-5.05,X-LC,10.64,213928.0,-2811.0,67473.0,-0.04,-1.30,31.54,29.84,6.0,-0.04,1.68,12.22,X40,ATH,ELECTRICAL


In [13]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,3.64,H-SC,100.00,119796.0,22150.0,25529.0,1.18,22.68,21.31,48.82,138.0,0.87,0.94,33.08,X40,ATH,PHARMA
4,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,91.49,99641.0,21279.0,12754.0,0.57,27.15,12.80,43.43,71.0,1.67,0.78,33.27,XY24,NTT,MISC
68,SBILIFE,1496.49,1924.99,1.57,H-LC,44.68,100621.0,16818.0,7174.0,-0.11,20.07,7.13,28.63,42.0,2.34,0.79,34.46,AR,ATH,INSURANCE
56,PIDILITIND,2504.06,3576.00,-17.12,X-LC,28.72,91905.0,16783.0,15376.0,1.05,22.34,16.73,42.81,11.0,1.09,0.72,15.26,X40,BTT,CHEMICALS


In [14]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,LUXIND,1448.19,2442.00,41.05,M-SC,77.66,146158.0,8580.0,85839.0,-2.95,6.24,58.73,68.62,182.0,0.10,1.15,26.71,X40N,NTT,TEXTILES
57,POLYCAB,5005.95,7541.35,6.73,H-MC,57.45,101320.0,16219.0,26880.0,1.30,19.06,26.53,50.65,92.0,0.60,0.80,27.36,X40N,ATH,CABLES
55,PFIZER,4245.50,6318.30,3.64,H-SC,100.00,119796.0,22150.0,25529.0,1.18,22.68,21.31,48.82,138.0,0.87,0.94,33.08,X40,ATH,PHARMA
28,GILLETTE,8109.44,10330.69,2.11,H-SC,71.28,150612.0,12752.0,25017.0,1.20,9.25,16.61,27.39,118.0,0.51,1.18,30.54,X40,ATH,FMCG
33,HEROMOTOCO,4311.81,6039.03,0.95,H-MC,85.11,153888.0,2975.0,57477.0,2.04,1.97,37.35,40.06,89.0,0.05,1.21,25.87,AR,ATH,AUTO


In [15]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
17,CAMS,3643.00,5250.99,-2.00,H-SC,22.34,110258.0,8254.0,36771.0,1.37,8.09,33.35,44.14,111.0,0.22,0.87,25.65,XR,ATH,MISC
11,BANKINDIA,113.49,190.00,-31.87,M-MC,15.96,133499.0,4347.0,82716.0,0.21,3.37,61.96,67.42,155.0,0.05,1.05,29.84,XR,NTT,BANKS
33,HEROMOTOCO,4311.81,6039.03,0.95,H-MC,85.11,153888.0,2975.0,57477.0,2.04,1.97,37.35,40.06,89.0,0.05,1.21,25.87,AR,ATH,AUTO
93,WIPRO,243.46,420.00,-9.71,M-LC,45.74,154566.0,3621.0,105831.0,0.78,2.40,68.47,72.51,69.0,0.03,1.21,16.38,XR,NTT,IT
38,INDIAMART,2327.09,4911.36,-52.59,H-SC,46.81,123410.0,74.0,136886.0,-1.67,0.06,110.92,111.05,108.0,0.00,0.97,20.92,AR,ATH,MISC


In [16]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,MASFIN,326.60,399.50,-21.86,H-SC,64.89,88680.0,-9300.0,31171.0,2.28,-9.49,35.15,22.32,130.0,-0.30,0.70,28.62,XR,ATH,FINANCE
88,VALIANTORG,512.64,838.00,-330.82,H-SC,78.72,98059.0,-35227.0,119818.0,0.03,-26.43,122.19,63.47,131.0,-0.29,0.77,64.37,XR,NTT,CHEMICALS
4,ADANIPORTS,1103.69,1583.00,-0.76,M-LC,91.49,99641.0,21279.0,12754.0,0.57,27.15,12.80,43.43,71.0,1.67,0.78,33.27,XY24,NTT,MISC
68,SBILIFE,1496.49,1924.99,1.57,H-LC,44.68,100621.0,16818.0,7174.0,-0.11,20.07,7.13,28.63,42.0,2.34,0.79,34.46,AR,ATH,INSURANCE
85,UJJIVANSFB,52.77,60.00,38.86,M-SC,36.17,120771.0,-21708.0,41231.0,-0.07,-15.24,34.14,13.70,217.0,-0.53,0.95,42.54,X40N,NTT,BANKS


In [17]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
55,PFIZER,4245.50,6318.30,3.64,H-SC,100.00,119796.0,22150.0,25529.0,1.18,22.68,21.31,48.82,138.0,0.87,0.94,33.08,X40,ATH,PHARMA
18,CERA,8421.60,10420.45,-17.90,X-SC,98.94,84682.0,-24799.0,50784.0,2.33,-22.65,59.97,23.73,64.0,-0.49,0.67,28.17,X40N,ATH,CERAMICS
78,TANLA,917.30,1963.11,-39.00,H-SC,97.87,149762.0,-89653.0,362604.0,0.63,-37.45,242.12,114.01,110.0,-0.25,1.18,37.35,AR,ATH,IT
74,SONACOMS,578.05,1006.00,-25.93,M-MC,96.81,95716.0,-5443.0,80334.0,1.80,-5.38,83.93,74.03,151.0,-0.07,0.75,34.52,AR,BTT,AUTO
24,DLF,706.18,1031.70,-15.67,H-LC,95.74,226180.0,20682.0,74051.0,0.10,10.06,32.74,46.10,30.0,0.28,1.78,27.83,AR,ATH,REALTY


In [18]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.45
1,25,40.37
2,50,69.38


In [19]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24    32.26
XY25    14.40
X40     14.20
X40N    13.71
AR      12.57
XR      10.28
X200     1.44
SR       1.14
Name: CurrAlloc%, dtype: float64

In [20]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    21.28
M-SC    18.07
X-LC    17.45
H-MC     8.71
M-LC     7.77
H-LC     7.27
X-MC     5.66
M-MC     5.60
X-SC     2.91
L-SC     2.31
L-LC     1.17
Name: CurrAlloc%, dtype: float64

In [21]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,13.12,-0.68,32.27
IT,10.00,-17.64,81.94
BANKS,9.56,-15.17,68.11
MISC,6.52,-30.38,88.04
FINANCE,6.03,-24.73,83.95
INSURANCE,6.02,1.90,27.36
PAINTS,5.47,-18.29,53.29
ELECTRICAL,4.81,-4.72,56.85
AUTO,4.56,-7.81,54.91


In [22]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2871185.0,26
AR,1246214.0,13
X40N,1203433.0,17
XR,1144170.0,14
X40,752545.0,11
XY25,471050.0,10
SR,199971.0,2
X200,75278.0,1


In [23]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,2311935.0,21
M-SC,2018062.0,21
X-LC,756313.0,12
M-MC,554729.0,5
H-MC,520081.0,8
X-SC,339620.0,4
M-LC,315779.0,6
X-MC,314628.0,5
L-SC,303242.0,4


In [24]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      921910.0      6
M-SC       XY24      749621.0      6
H-SC       AR        558887.0      3
           X40N      503186.0      6
M-SC       X40N      463186.0      6
X-LC       X40       425219.0      5
M-MC       XY24      391679.0      3
M-SC       XR        294511.0      4
H-SC       XR        277406.0      4
H-LC       AR        251043.0      4
M-SC       X40       213517.0      2
M-LC       XY24      191470.0      4
H-MC       XY24      183296.0      2
L-SC       XR        156269.0      2
X-LC       AR        140821.0      2
X-MC       XY24      139686.0      2
M-SC       XY25      119404.0      1
X-SC       SR        115529.0      1
X-LC       XY25      114995.0      4
H-MC       XR        114574.0      1
X-SC       XR        112863.0      1
           X40N      111228.0      2
M-LC       XR        105831.0      1
X-MC       X40N       98953.0      2
M-SC       AR         93381.0      1
           SR         84442.0      1
M-MC       XR         82716.0      1
L-SC       XY24       82702.0      1
M-MC       AR         80334.0      1
X-MC       XY25       75989.0      1
X-LC       X200       75278.0      1
H-MC       XY25       74591.0      1
L-SC       AR         64271.0      1
H-MC       X40        63263.0      2
           AR         57477.0      1
H-SC       X40        50546.0      2
L-LC       XY25       40121.0      1
H-LC       XY25       27472.0      1
H-MC       X40N       26880.0      1
M-LC       XY25       18478.0      1

In [25]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 57.0 seconds
